In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/input/datavidia2019/flight.csv')
titanic = pd.read_csv('/kaggle/input/titanic/train.csv')

# PREPROCESSING

## Preprocessing: outliers
Using CLIP 

In [ ]:
UPPERBOUND, LOWERBOUND = np.percentile(train.price,[99,1])

In [ ]:
y = np.clip(train.price,UPPERBOUND, LOWERBOUND)

In [ ]:
train['price_clipped'] = y

In [ ]:
train.price.hist(bins=50)

In [ ]:
train.price_clipped.hist(bins=50)

## Preprocessing: rank

In [ ]:
sr = pd.Series([10, 25, 3, 11, 24, 6]) 

In [ ]:
index_ = ['Coca Cola', 'Sprite', 'Coke', 'Fanta', 'Dew', 'ThumbsUp'] 

In [ ]:
sr.index = index_ 

In [ ]:
print(sr)

In [ ]:
result = sr.rank() 

In [ ]:
result

In [ ]:
train.airlines_name.value_counts()

In [ ]:
train.airlines_name.replace(['6c483c0812c96f8ec43bb0ff76eaf716','33199710eb822fbcfd0dc793f4788d30', '0a102015e48c1f68e121acc99fca9a05',
                             'ad5bef60d81ea077018f4d50b813153a', '74c5549aa99d55280a896ea50068a211','e35de6a36d385711a660c72c0286154a'
                                ,'9855a1d3de1c46526dde37c5d6fb758c','6872b49542519aea7ae146e23fab5c08'],
                            ['t1','t2','t3','t4','t5','t6','t7','t8'], inplace=True)

In [ ]:
train.head()

In [ ]:
train_ = train[['airlines_name', 'price']]

In [ ]:
train_.head(3)

In [ ]:
train['price_rank'] = train_.rank().price

In [ ]:
a = pd.Series([100,1,10])
a.rank()

## Preprocessing on Categorical and Ordinal Features
- How to preproccess categorical and ordinal features ?
- How to generate new features from them ?

## Preprocessing: Label Encoder

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
train.head(1)

In [ ]:
le.fit(train.trip)

In [ ]:
le.classes_

In [ ]:
trip_label = le.transform(train.trip)

In [ ]:
train['trip_label'] = trip_label

In [ ]:
train.trip_label.value_counts()

In [ ]:
train.trip.value_counts()

## Preprocessing: Factorize

In [ ]:
labels, uniques = pd.factorize(train.trip) 

In [ ]:
train['trip_factorize'] = labels

In [ ]:
train.trip_factorize.value_counts()

## Preprocessing: Frequency Encoding

In [ ]:
encoding = train.groupby('trip').size()
encoding = encoding/len(train)
train['enc']= train.trip.map(encoding)

In [ ]:
train.enc.value_counts()

## Interactions of categorical features

In [ ]:
# Example of interaction of categorical.
train['gender_trip'] = train['gender'] + train['trip']

In [ ]:
train.gender_trip.head()

### SUMMARY
1. Values in ordinal features are sorted in some meaningful order
2. Label encoding maps categories to numbers
3. Frequency encoding maps categories to their frequencies
4. Label and frequency encodings are often used for tree-based models
5. One-hot encoding is often used for non-tree based models
6. Interactions of categorical features can help linear models and KNN

# Datetime and Coordinates
- Haven't watched yet

# Handling Missing Values

### Visualizing Missing Values !
- Used to find pattern of occurence of missing values. This is important because there is something that makes missing values has pattern.

In [ ]:
import missingno as msno

In [ ]:
# Plot Missing Values in Datavidia 19 Data
sns.heatmap(train.isnull(), cbar=False)

In [ ]:
# Plot Missing Values in Titanic Data
sns.heatmap(titanic.isnull(), cbar=False)

In [ ]:
# Plot Missing Values in Datavidia 19 Data
msno.matrix(train)

In [ ]:
# Plot Missing Values in Titanic Data
msno.matrix(titanic)

The bar on the right side, shows that this is a line plot for each row's data completeness. In this dataset, all rows have 10-12 valid values and
hence 0-2 missing values.

In [ ]:
msno.heatmap(titanic)

In [ ]:
msno.bar(titanic)

In [ ]:
msno.dendrogram(titanic)

- Dapat dilihat, semakin lebar jarak antara dua kolom maka semakin tidak mirip kedua kolom tersebut. Semakin dekat jarak antara dua kolom semakin mirip kedua kolom tersebut.
- Dari dataset diatas, dapat dibedakan menjadi 2, yaitu Cabin dan yang bukan Cabin.

### Hidden NaNs
- How to find hidden NaNs ? Hidden means the missing values (NaN) replaced by -1, -999, or etc.

### Fillna approaches
1. -999, -1, etc
2. mean median
3. reconstruct value

- Note that linear model can suffer because fillna

### "Isnull" Feature
- Tree and Neural Network can be beneficial while imputing mean or median.

In [ ]:
# Mencatat Indexnya
index = 0

# Membuat Kolom isnull
titanic['isnull_'] = False

for i in titanic.Age.isnull():
    if i == True:
        titanic.isnull_[index] = i
    index += 1
    
titanic.tail()

# After making the isnull feature, imput the NaN with median or mean value.

### Missing Value Reconstruction
- There is one general concern, if you do the feature generation with missing values column.
- Note : Be careful impute the missing values before the feature generation.
- The difference between today and yesterday (missing value will) in timeseries is big, if you do the imputation with median or mean.
- Interpolate the missing values in time series data ! [ But usually we don't have time to do carefully this ]

### Feature Generation with missing values

In [ ]:
row = [['A',1], ['A',4], ['A',2], ['A',-1], ['B',9], ['B', np.nan]]
df = pd.DataFrame(row, columns = ['categorical_feature','numeric_feature'])
df

In [ ]:
df['numeric_feature_filled'] = df.numeric_feature.fillna(-999)
df

In [ ]:
# Membuat kolom categorical_encoded : 
# 1. Jumlah total dari numeric_feature_filled yang diklasifikasikan oleh categorical_encoded dibagi dengan jumlah total categorical_feature.
# 2. Mapping nilai tersebut untuk setiap nilai categorical_feature.
df['categorical_encoded'] = df.categorical_feature.map(df.groupby('categorical_feature').numeric_feature_filled.sum() / df['categorical_feature'].value_counts())
df

Important note that this imputation method will screw up our model, so to handle this is ignore the mean, median or missing values when calculating the categorical_encoded feature.

Remember ! Carefully early nan imputation if you do the feature generation.
XGBoost can handle missing values and it will drastically increase the score.



In [ ]:
# When you calculating the categorical_encoded without missing values's value.
df['numeric_feature'] = df.numeric_feature.fillna(0)
df['categorical_appropriate'] = df.categorical_feature.map(df.groupby('categorical_feature').numeric_feature.sum()/ df['categorical_feature'].value_counts())
df

**Sometimes we can treat outlier as missing values.**

### Treating values which do not present in train data

In [ ]:
row_train = [['A',0], ['A',1], ['A',1], ['A',1], ['B',0], ['B', 0], ['D',1]]
row_test = ['A','A','B','C']
tes_train = pd.DataFrame(row_train, columns = ['categorical_feature','target'])
tes_test = pd.DataFrame(row_test, columns = ['categorical_feature'])

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
display_side_by_side(tes_train, tes_test)

In [ ]:
e = pd.DataFrame([[6,'A'],[3,'B'],[1,'D']], columns = ['Categorical_e','index'])
e_ = pd.DataFrame([[6,'A'],[3,'B'],[1,'C']], columns = ['Categorical_e','index'])

In [ ]:
# Create number of occurences in the data for categorical_feature column
tes_train['categorical_encoded'] = tes_train.categorical_feature.map(e.groupby('index').Categorical_e.sum())
tes_test['categorical_encoded'] = tes_test.categorical_feature.map(e_.groupby('index').Categorical_e.sum())

In [ ]:
display_side_by_side(tes_train, tes_test)

We treat 'C' value in categorical_feature test set same as 'D' value in categorical_feature train set.

if there is a dependencies between target with a number of occurences in each category, our model will be succesfully utilized that.

#### SUMMARY
1. The choice of method to fill NaN depends on the situtation.
2. Usual way to deal with missing values is to replace them with -999, mean or median.
3. Missing values already can be replaced with something by organizers.
4. Binary feature 'isnull' can be beneficial.
5. In general, avoid filling NaNs before feature generation.
6. Xgboost can handle NaN.